In [1]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 教育版, 18363, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]'

'2020-10-04 23:48:13'

- **@author**: run_walker
- **@references**:
    1. [官方文档 > itertools](https://docs.python.org/zh-cn/3.7/library/itertools.html#module-itertools) 给出了每个函数的伪代码实现，以及使用现有的itertools作为基础构件来创建扩展的工具集
    2. [Python标准库13 循环器 (itertools)](https://www.cnblogs.com/vamei/p/3174796.html)
    3. [廖雪峰的官方网站 -> Python教程 -> 常用内建模块 -> itertools](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/00143200162233153835cfdd1a541a18ddc15059e3ddeec000)

In [4]:
import itertools

pdir(itertools)

module attribute:
    __loader__, __name__, __package__, __spec__
special attribute:
    __doc__
class:
    _grouper: 
    _tee: Iterator wrapped to make it copyable
    _tee_dataobject: Data container common to multiple tee objects.
    accumulate: accumulate(iterable[, func]) --> accumulate object
    chain: chain(*iterables) --> chain object
    combinations: combinations(iterable, r) --> combinations object
    combinations_with_replacement: combinations_with_replacement(iterable, r) --> combinations_with_replacement object
    compress: compress(data, selectors) --> iterator over selected data
    count: count(start=0, step=1) --> count object
    cycle: cycle(iterable) --> cycle object
    dropwhile: dropwhile(predicate, iterable) --> dropwhile object
    filterfalse: filterfalse(function or None, sequence) --> filterfalse object
    groupby: groupby(iterable, key=None) -> make an iterator that returns consecutive
    islice: islice(iterable, stop) --> islice object
    permutati

In [2]:
from itertools import *

# 无限迭代器
Infinite Iterators

|Iterator	|Arguments|	Results	|Example|
|:--|:--:|-----|-----|
|count()	|start, [step]|	start, start+step, start+2*step, …	|count(10) --> 10 11 12 13 14 ...|
|cycle()	|p|	p0, p1, … plast, p0, p1, …	|cycle('ABCD') --> A B C D A B C D ...|
|repeat()	|elem, [n]|	elem, elem, elem, … endlessly or up to n times	|repeat(10, 3) --> 10 10 10|

<div class="alert alert-block alert-warning">
    <i class="fa fa-sticky-note" aria-hidden="true"><b> Note:</b></i>
    使用无限迭代器的时候，只能通过`next`一个个取出，千万不能直接用`list`，无穷无尽，没有停止条件的话，内存会爆。
</div>

## count()

In [10]:
?count

Init signature: count(self, /, *args, **kwargs)
Docstring:     
count(start=0, step=1) --> count object

Return a count object whose .__next__() method returns consecutive values.
Equivalent to:

    def count(firstval=0, step=1):
        x = firstval
        while 1:
            yield x
            x += step
Type:           type
Subclasses:     


In [6]:
# 例1 不指定起点时默认从0开始，不指定间隔时默认为1
a = count()
type(a)
a.__next__()
a.__next__()

itertools.count

0

1

In [7]:
# 例2 
a = count(4, -2)
a.__next__()
a.__next__()
a.__next__()
a.__next__()

4

2

0

-2

In [8]:
# 例3
a = map(lambda x: x**2, count())
type(a)
a.__next__()
a.__next__()
a.__next__()

map

0

1

4

## cycle()

In [11]:
?cycle

Init signature: cycle(self, /, *args, **kwargs)
Docstring:     
cycle(iterable) --> cycle object

Return elements from the iterable until it is exhausted.
Then repeat the sequence indefinitely.
Type:           type
Subclasses:     


In [12]:
# 例1 list
b = cycle([1, 2])
type(b)
for _ in range(6):
    print(b.__next__())

itertools.cycle

1
2
1
2
1
2


In [13]:
# 例2 str
b = cycle('syhd')
for _ in range(8):
    print(b.__next__())

s
y
h
d
s
y
h
d


## repeat()

In [14]:
?repeat

Init signature: repeat(self, /, *args, **kwargs)
Docstring:     
repeat(object [,times]) -> create an iterator which returns the object
for the specified number of times.  If not specified, returns the object
endlessly.
Type:           type
Subclasses:     


In [15]:
# 例1
c = repeat('zyyhhzgwm')
type(c)
c.__next__()
c.__next__()

itertools.repeat

'zyyhhzgwm'

'zyyhhzgwm'

<font color="red">没看出来有什么意义啊，直接设定一个字符串常量不行吗？</font>

In [16]:
# 例2 指定有限的重复次数
c = repeat('smcmywgyx', 2)

c.__next__()
c.__next__()

try:
    c.__next__()
except Exception as e:
    e

'smcmywgyx'

'smcmywgyx'

StopIteration()

## takewhile()
* 无限序列虽然可以无限迭代下去，但是通常我们会通过`takewhile()`等函数根据条件判断来截取出一个有限的序列。
* 当函数返回True时，收集元素到循环器；一旦函数返回False，则停止。

In [17]:
num = cycle(range(10))
list(takewhile(lambda x: x <= 4, num))

[0, 1, 2, 3, 4]

# 终止于最短输入序列的迭代器
Iterators terminating on the shortest input sequence

|	Iterator	|	Arguments	|	Results	|	Example	|
|	:--	|	:--	|	:--	|	:--	|
|	accumulate()	|	p, [func]	|	p0, p0+p1, p0+p1+p2, …	|	accumulate([1,2,3,4,5]) --> 1 3 6 10 15	|
|	chain()	|	p, q, …	|	p0, p1, … plast, q0, q1, …	|	chain('ABC', 'DEF') --> A B C D E F	|
|	chain.from_iterable()	|	iterable	|	p0, p1, … plast, q0, q1, …	|chain.from_iterable(['ABC', 'DEF']) --> A B C D E F|
|	compress()	|	data, selectors	|	(d[0] if s[0]), (d[1] if s[1]), …	|	compress('ABCDEF', [1,0,1,0,1,1]) --> A C E F	|
|	dropwhile()	|	pred, seq	|	seq[n], seq[n+1], starting when pred fails	|	dropwhile(lambda x: x<5, [1,4,6,4,1]) --> 6 4 1	|
|	filterfalse()|pred, seq	|elements of seq where pred(elem) is false|filterfalse(lambda x: x%2, range(10)) --> 0 2 4 6 8	|
|	groupby()	|	iterable[, keyfunc]	|	sub-iterators grouped by value of keyfunc(v)	|		|
|	islice()|	seq, [start,] stop, [step]	|	elements from seq[start:stop:step]	|	islice('ABCDEFG', 2, None) --> C D E F G	|
|	starmap()	|	func, seq	|	func(*seq[0]), func(*seq[1]), …	|	starmap(pow, [(2,5), (3,2), (10,3)]) --> 32 9 1000	|
|	takewhile()	|	pred, seq	|	seq[0], seq[1], until pred fails	|	takewhile(lambda x: x<5, [1,4,6,4,1]) --> 1 4	|
|	tee()	|	it, n	|	it1, it2, … itn splits one iterator into n	|		|
|	zip_longest()	|	p, q, …	|	(p[0], q[0]), (p[1], q[1]), …	|	zip_longest('ABCD', 'xy', fillvalue='-') --> Ax By C-D-	|


## accumulate()
前n项和

In [3]:
?accumulate

Init signature: accumulate(self, /, *args, **kwargs)
Docstring:     
accumulate(iterable[, func]) --> accumulate object

Return series of accumulated sums (or other binary function results).
Type:           type
Subclasses:     


In [4]:
# 例1
list(accumulate(range(1, 6)))

[1, 3, 6, 10, 15]

$[a_i, ...], 1\leq i\leq n$，则$b_1=a_1, b_i=f(b_{i-1}, a_i)$

In [5]:
# 例2
list(accumulate(range(1, 6), lambda x, y: x**2 + y))

[1, 3, 12, 148, 21909]

In [6]:
al = list(range(1, 6))
bl = [al[0]]
f = lambda x, y: x**2 + y
for i in range(1, len(al)):
    bl.append(f(bl[-1], al[i]))
bl

[1, 3, 12, 148, 21909]

In [41]:
# 例3
from operator import mul

data = [3, 4, 6, 2, 1, 9, 0, 7, 5, 8]
list(accumulate(data, mul)) 
list(accumulate(data, min))
list(accumulate(data, max))

[3, 12, 72, 144, 144, 1296, 0, 0, 0, 0]

[3, 3, 3, 2, 1, 1, 0, 0, 0, 0]

[3, 4, 6, 6, 6, 9, 9, 9, 9, 9]

## chain()
把一组可迭代对象串联起来，形成一个更大的迭代器

In [8]:
# 例1
list(chain('abc', 'def'))

['a', 'b', 'c', 'd', 'e', 'f']

In [9]:
# 例2
list(chain('ab', 'cd', 'ef'))

['a', 'b', 'c', 'd', 'e', 'f']

## chain.from_iterable() 

In [10]:
list(chain.from_iterable(['ABC', 'DEF']))

['A', 'B', 'C', 'D', 'E', 'F']

In [11]:
list(chain.from_iterable(['ab', 'cd', 'ef']))

['a', 'b', 'c', 'd', 'e', 'f']

## groupby()
* 把迭代器中**相邻的**重复元素挑出来放在一起
* 实际上挑选规则是通过函数完成的，只要作用于函数的两个元素返回的值相等，这两个元素就被认为是在一组的，而函数返回值作为组的key

In [15]:
?groupby

Init signature: groupby(self, /, *args, **kwargs)
Docstring:     
groupby(iterable, key=None) -> make an iterator that returns consecutive
keys and groups from the iterable.  If the key function is not specified or
is None, the element itself is used for grouping.
Type:           type
Subclasses:     


In [20]:
# 例1
for key, group in groupby('AaaBBbcCAAa'):
    print(key, list(group))

A ['A']
a ['a', 'a']
B ['B', 'B']
b ['b']
c ['c']
C ['C']
A ['A', 'A']
a ['a']


In [21]:
# 例2
for key, group in groupby('AaaBBbcCAAa', lambda c: c.upper()):
    print(key, list(group))

A ['A', 'a', 'a']
B ['B', 'B', 'b']
C ['c', 'C']
A ['A', 'A', 'a']


## starmap()

In [16]:
?starmap

Init signature: starmap(self, /, *args, **kwargs)
Docstring:     
starmap(function, sequence) --> starmap object

Return an iterator whose values are returned from the function evaluated
with an argument tuple taken from the given sequence.
Type:           type
Subclasses:     


In [17]:
map_obj = map(pow, [1, 2, 3], [1, 2, 3])
starmap_obj = starmap(pow, [(1, 1), (2, 2), (3, 3)])

type(map_obj)
type(starmap_obj)

map

itertools.starmap

In [18]:
list(map_obj)
list(starmap_obj)

[1, 4, 27]

[1, 4, 27]

In [22]:
list(map(pow, *zip(*[(1, 1), (2, 2), (3, 3)])))
list(starmap(pow, zip([1, 2, 3], [1, 2, 3])))

[1, 4, 27]

[1, 4, 27]

`map`和`starmap`作用类似，只是传参的方式不同，可以通过`zip`等价实现。

## zip_longest()

In [23]:
?zip_longest

Init signature: zip_longest(self, /, *args, **kwargs)
Docstring:     
zip_longest(iter1 [,iter2 [...]], [fillvalue=None]) --> zip_longest object

Return a zip_longest object whose .__next__() method returns a tuple where
the i-th element comes from the i-th iterable argument.  The .__next__()
method continues until the longest iterable in the argument sequence
is exhausted and then it raises StopIteration.  When the shorter iterables
are exhausted, the fillvalue is substituted in their place.  The fillvalue
defaults to None or can be specified by a keyword argument.
Type:           type
Subclasses:     


In [24]:
list(zip_longest(range(3), range(6), fillvalue='*'))

[(0, 0), (1, 1), (2, 2), ('*', 3), ('*', 4), ('*', 5)]

# 排列组合迭代器
Combinatoric generators

|Iterator|Arguments|Results|
|:--|:--|:--|
|product()|p, q, … [repeat=1]|cartesian product, equivalent to a nested for-loop|
|permutations()|p[, r]|r-length tuples, all possible orderings, no repeated elements|
|combinations()|p, r|r-length tuples, in sorted order, no repeated elements|
|combinations_with_replacement()|p, r|r-length tuples, in sorted order, with repeated elements|
|product('ABCD', repeat=2)| |AA AB AC AD BA BB BC BD CA CB CC CD DA DB DC DD|
|permutations('ABCD', 2)| |AB AC AD BA BC BD CA CB CD DA DB DC|
|combinations('ABCD', 2)| |AB AC AD BC BD CD|
|combinations_with_replacement('ABCD', 2)| |AA AB AC AD BB BC BD CC CD DD|

## product

In [25]:
list(product('ABCD', 'xy'))

[('A', 'x'),
 ('A', 'y'),
 ('B', 'x'),
 ('B', 'y'),
 ('C', 'x'),
 ('C', 'y'),
 ('D', 'x'),
 ('D', 'y')]

In [26]:
list(product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

## permutations()
排列，$A_n^m, m\leq n$

In [28]:
?permutations

Init signature: permutations(self, /, *args, **kwargs)
Docstring:     
permutations(iterable[, r]) --> permutations object

Return successive r-length permutations of elements in the iterable.

permutations(range(3), 2) --> (0,1), (0,2), (1,0), (1,2), (2,0), (2,1)
Type:           type
Subclasses:     


In [27]:
list(permutations(range(4)))

[(0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 2, 1, 3),
 (0, 2, 3, 1),
 (0, 3, 1, 2),
 (0, 3, 2, 1),
 (1, 0, 2, 3),
 (1, 0, 3, 2),
 (1, 2, 0, 3),
 (1, 2, 3, 0),
 (1, 3, 0, 2),
 (1, 3, 2, 0),
 (2, 0, 1, 3),
 (2, 0, 3, 1),
 (2, 1, 0, 3),
 (2, 1, 3, 0),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (3, 0, 1, 2),
 (3, 0, 2, 1),
 (3, 1, 0, 2),
 (3, 1, 2, 0),
 (3, 2, 0, 1),
 (3, 2, 1, 0)]

In [35]:
list(permutations(range(4), 1))  # 得到的总是tuple，即便长度为1

[(0,), (1,), (2,), (3,)]

In [29]:
list(permutations(range(4), 2))

[(0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2)]

## combinations
组合，$C_n^m, m\leq n$

In [34]:
?combinations

Init signature: combinations(self, /, *args, **kwargs)
Docstring:     
combinations(iterable, r) --> combinations object

Return successive r-length combinations of elements in the iterable.

combinations(range(4), 3) --> (0,1,2), (0,1,3), (0,2,3), (1,2,3)
Type:           type
Subclasses:     


In [33]:
list(combinations(range(4), 1))

[(0,), (1,), (2,), (3,)]

In [30]:
list(combinations(range(4), 2))

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

In [31]:
list(combinations(range(4), 3))

[(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]

In [32]:
list(combinations(range(4), 4))

[(0, 1, 2, 3)]

## combinations_with_replacement()
带放回的组合

In [36]:
list(combinations_with_replacement(range(4), 2))

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 2),
 (2, 3),
 (3, 3)]

In [37]:
list(combinations_with_replacement(range(4), 1))

[(0,), (1,), (2,), (3,)]